In [1]:
import pandas as pd

In [39]:
df = pd.read_csv('unified_df.csv')
df = df.drop(columns=[c for c in df.columns if 'Unnamed' in c])
df

ParserError: Error tokenizing data. C error: EOF inside string starting at row 65514

# Accuracy

In [30]:
mean_acc = df.groupby(['dataset', 'filtering_algorithm', 'n_selected_features']).mean()['test_acc'].reset_index()

max_acc = mean_acc.groupby('dataset').max()['test_acc'].reset_index()

max_acc = max_acc.apply(
    lambda row: mean_acc[(mean_acc['dataset'] == row['dataset']) & (mean_acc['test_acc'] == row['test_acc'])].iloc[0],
    axis=1)
max_acc

,dataset,filtering_algorithm,n_selected_features,test_acc
0,ALL.csv,grey_wolf_fs,4.0,0.799744
1,ALLAML.csv,rfe_svm_fs,25.0,1.000000
2,BASEHOCK.csv,mrmr_fs,100.0,0.951636
3,Breast.csv,svm_fs_New,100.0,0.975258
4,CLL-SUB-111.csv,poly_svm_fs_New,100.0,0.822121
5,CLL.csv,rfe_svm_fs,100.0,1.000000
6,CNS.csv,svm_fs_New,20.0,0.943333
7,COIL20.csv,svm_fs,100.0,0.967639
8,FSH.csv,svm_fs,100.0,0.800000
9,Lung.csv,rfe_svm_fs,50.0,0.918947


# Fit time

In [47]:
df[df['dataset'] == 'ALL.csv'].groupby(['filtering_algorithm'])['test_feature_selector_fit_time'].max()

filtering_algorithm
grey_wolf_fs        6536.657198
grey_wolf_fs_New    9969.911747
mrmr_fs             2325.742276
poly_svm_fs         6545.411394
rbf_svm_fs          3381.926456
rbf_svm_fs_New         4.500997
reliefF_fs             6.942051
rfe_svm_fs          2834.276628
select_fdr_fs          0.183367
svm_fs              3670.037053
svm_fs_New             4.964416
Name: test_feature_selector_fit_time, dtype: float64

In [50]:
metric_col = 'test_feature_selector_fit_time'
mean_acc = df.groupby(['dataset', 'filtering_algorithm']).max()[metric_col].reset_index()

max_acc = mean_acc.groupby(['dataset']).min()[metric_col].reset_index()

max_acc = max_acc.apply(lambda row: mean_acc[(mean_acc['dataset'] == row['dataset'])
                                             & (mean_acc[metric_col] == row[metric_col])].iloc[0], axis=1)
max_acc

C:\Users\adutl\AppData\Local\Temp\ipykernel_10932\877760687.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  mean_acc = df.groupby(['dataset', 'filtering_algorithm']).max()[metric_col].reset_index()


,dataset,filtering_algorithm,test_feature_selector_fit_time
0,ALL.csv,select_fdr_fs,0.183367
1,ALLAML.csv,select_fdr_fs,0.075487
2,BASEHOCK.csv,select_fdr_fs,1.473152
3,Breast.csv,select_fdr_fs,0.189027
4,CLL-SUB-111.csv,select_fdr_fs,0.091319
5,CLL.csv,select_fdr_fs,0.025800
6,CNS.csv,select_fdr_fs,0.046296
7,COIL20.csv,select_fdr_fs,0.105482
8,FSH.csv,select_fdr_fs,0.012692
9,Lung.csv,select_fdr_,0.197585
